In [ ]:
from statsmodels.stats import inter_rater as irr

In [ ]:
import pandas as pd

annot_1 = set(pd.read_excel("terms_nicola.xlsx")["form"].values)
annot_2 = set(pd.read_excel("terms_michele.xlsx")["form"].values)

tp = len(annot_1.intersection(annot_2))
fp = len(annot_1.difference(annot_2))
fn = len(annot_2.difference(annot_1))

p = tp / (tp + fp)
r = tp / (tp + fn)

f =  2 * r * p / (r + p)
print(f)

In [ ]:
import pandas as pd

annot_1 = set(pd.read_excel("terms_nicola_corretti.xlsx")["form"].values)
annot_2 = set(pd.read_excel("terms_michele_corretti.xlsx")["form"].values)

tp = len(annot_1.intersection(annot_2))
fp = len(annot_1.difference(annot_2))
fn = len(annot_2.difference(annot_1))

p = tp / (tp + fp)
r = tp / (tp + fn)

f =  2 * r * p / (r + p)
print(f)

In [ ]:
import pandas as pd
forms = set(pd.read_excel("terms_corretti.xlsx")["form"].values)
subs = pd.read_excel("terms_corretti.xlsx")[["form", "subterms"]]
subs = subs.fillna("")
subs = {row.form : row.subterms.split("; ") for row in subs.itertuples() if row.subterms}

In [ ]:
from terms2lod import terms2lod
import json
import codecs
from multiset import Multiset
import numpy as np
from itertools import chain

def get_entities_from_doc(doc):
    text = doc["text"]
    ents = terms2lod.get_entities(doc["entities"], text)
    ents = terms2lod.join_ents(ents, doc["relations"])
    ents = terms2lod.only_longest(ents)
    return [e[0] for e in ents.values()]

with codecs.open("admin.jsonl", "r", "utf8") as filein:
    json_list = list(filein)
corpus_1 = [json.loads(json_str) for json_str in json_list]

with codecs.open("michele.jsonl", "r", "utf8") as filein:
    json_list = list(filein)
corpus_2 = [json.loads(json_str) for json_str in json_list] 

values = list()
all_1 = list()
all_2 = list()

for doc_1, doc_2 in zip(corpus_1, corpus_2):

    e1 = get_entities_from_doc(doc_1)
    e2 = get_entities_from_doc(doc_2)

    all_1 +=  e1 + list(chain.from_iterable([subs[e] for e in e1 if e in subs]))
    all_2 += e2 + list(chain.from_iterable([subs[e] for e in e2 if e in subs]))

    annot_1 = Multiset(get_entities_from_doc(doc_1))
    annot_2 = Multiset(get_entities_from_doc(doc_2))

    tp = len(annot_1.intersection(annot_2))
    fp = len(annot_1.difference(annot_2))
    fn = len(annot_2.difference(annot_1))
    try:
        p = tp / (tp + fp)
        r = tp / (tp + fn)
        f =  2 * r * p / (r + p)
    except ZeroDivisionError:
        f = 0
    values.append(f)


annot_1 = Multiset([t for t in all_1 if t in forms])
annot_2 = Multiset([t for t in all_2 if t in forms])

tp = len(annot_1.intersection(annot_2))
fp = len(annot_1.difference(annot_2))
fn = len(annot_2.difference(annot_1))

p = tp / (tp + fp)
r = tp / (tp + fn)
f =  2 * r * p / (r + p)
print(f)

In [9]:
import pandas as pd
df_1 = pd.read_excel("terms_nicola_corretti.xlsx")
df_2 = pd.read_excel("terms_michele_corretti.xlsx")

forms_1 = set(df_1["form"].values)
forms_2 = set(df_2["form"].values)
common = forms_1.intersection(forms_2)

df_1 = df_1[df_1["form"].isin(common)][["form", "domain"]]
df_2 = df_2[df_2["form"].isin(common)][["form", "domain"]]

df_3 = df_1.merge(df_2, on="form", how="left")
domain2idx = {
    "Law": 0,
    "EU Law": 1,
    "Waste management": 2,
    "Waste management law": 3,
    "Environment": 4,
    "Other": 5
}
tags = list()
for row in df_3.itertuples():
    l = [0, 0, 0, 0, 0, 0]
    l[domain2idx[row.domain_x]] += 1
    l[domain2idx[row.domain_y]] += 1
    tags.append(l)

In [16]:
from statsmodels.stats.inter_rater import fleiss_kappa
fleiss_kappa(tags)

0.7070998259627829